# II. Base Containers

In [1]:
import sys

sys.path.append("../../forme-groups-python-3-12/")

## 1. Create a Base Container

In [2]:
from src.groups.base.container import BaseContainer

container_hello = BaseContainer({"hello": "world"}, "dictionary")
print(f'Representation: {repr(container_hello)}')
print(f'String: {container_hello}')

Representation: BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=dict)
String: {'hello': 'world'}


Note: Base Containers are immutable. You cannot change the base container once it is created.

In [3]:
from attrs.exceptions import FrozenInstanceError

# Check that the value is frozen
try:
    container_hello._items = 'world2'
except FrozenInstanceError as e:
    print(f'Error: {e.msg}')

Error: can't set attribute


## 2. Base Container Types

Base Containers support the following types:
- **Dictionary** - A dictionary of key-value pairs.
- **List** - A list of values.
- **Tuple** - A tuple of values.  An immutable list.  Item order is preserved.
- **Set** - A set of values.  Sets contain unique values.  No values are duplicated.
- **Frozen Set** - A frozen set of values.  Frozen sets contain unique values.  No values are duplicated.


In [4]:
container_dict = BaseContainer({"hello": "world"}, "dictionary")
print(f'{repr(container_dict)}')

container_list = BaseContainer(["hello", "world"], "list")
print(f'{repr(container_list)}')

container_tuple = BaseContainer(("hello", "world"), "tuple")
print(f'{repr(container_tuple)}')

container_set = BaseContainer({"hello", "world"}, "set")
print(f'{repr(container_set)}')

container_frozenset = BaseContainer(frozenset({"hello", "world"}), "frozenset")
print(f'{repr(container_frozenset)}')

BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=dict)
BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=list)
BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=tuple)
BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=set)
BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=frozenset)


## 2A. Forcing Base Container Types

You can force a base container to be a specific type when it is created.

Items are stored in a container by deffault as Tuple of Base Values.  

In [5]:
container_dict = BaseContainer(("hello", "world"), "dictionary")
print(f'{repr(container_dict)}')
print(f'Container Type: {container_dict.type}')
print(f'Container Value: {container_dict}')

BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=dict)
Container Type: dict
Container Value: {'hello': 'world'}


## 3. Hashing Base Containers

Base Containers are hashed using the SHA-256 algorithm.  This is a one-way hash.  You cannot reverse the hash to get the original value.

Base Containers can be hashed a numbebr of ways:
- **BaseContainer._hash_repr()** - Hashes the container's representation.
- **BaseContainer._hash_items()** - Hashes the container's items values as leaves of a merkle tree.
- **BaseConntainer._hash_type()** - Hashes the container's type.
- **BaseContainer._hash()** - Returns the hash value of the container.

In [6]:
# BaseContainer({"hello": "world"}, "dictionary")
print(f'Representation of String Value: {repr(container_dict)}')
print(f'Representation Hash: {container_dict._hash_repr()}')
print(f'Items Hash: {container_dict._hash_items()}')
print(f'Type Hash: {container_dict._hash_type()}')
print(f'Hash: {container_dict._hash()}')

Representation of String Value: BaseContainer(items=(BaseValue(value='hello', type=str), BaseValue(value='world', type=str)), type=dict)
Representation Hash: fb06c1e2d2522990bc8de3b3fd3edfd97f29a651e834a9373c38fedb9dfc9051
Items Hash: 470573df702151426e45ae5664caf4a7c0a46b2fff6c0321fede8bd2cba68bf1
Type Hash: 6ab47d70854a8c690a0c2035be903f3d812cbab06f9e442e9b10ad70b1acd446
Hash: 246fdadf57904d780ed5f498fabd74efdf08c83a2c92a6c64f829020f534faf2


## 4. Verify Base Container Hashes

Below are some examples of how to verify the hash of a base container.

In [7]:
container_one = BaseContainer({"hello": "world"}, "dictionary")
container_two = BaseContainer({"hello": "worlds"}, "dictionary")

print(f'Container One items hash: {container_one._hash_items().root()}')
print(f'Container Two items hash: {container_two._hash_items().root()}')

if container_one._hash_items().root() != container_two._hash_items().root():
    print('Hashes are not equal. 👍')

print(f'Container One type hash: {container_one._hash_type()}')
print(f'Container Two type hash: {container_two._hash_type()}')

if container_one._hash_type() == container_two._hash_type():
    print('Hashes are equal. 👍')

print(f'Container One hash: {container_one._hash()}')
print(f'Container Two hash: {container_two._hash()}')

if container_one._hash().root() != container_two._hash().root():
    print('Hashes are not equal. 👍')

Container One items hash: 470573df702151426e45ae5664caf4a7c0a46b2fff6c0321fede8bd2cba68bf1
Container Two items hash: a84e2ef29482712f22ea4f5cb6d95832d561ab87aa6209f9d97ab7e84e8309a5
Hashes are not equal. 👍
Container One type hash: 6ab47d70854a8c690a0c2035be903f3d812cbab06f9e442e9b10ad70b1acd446
Container Two type hash: 6ab47d70854a8c690a0c2035be903f3d812cbab06f9e442e9b10ad70b1acd446
Hashes are equal. 👍
Container One hash: 246fdadf57904d780ed5f498fabd74efdf08c83a2c92a6c64f829020f534faf2
Container Two hash: 24fc1fbcf28c51a2860d88d92796e64acf97da2e740da5b8e3d34d9a87a1a296
Hashes are not equal. 👍


The function **BaseContainer._verify_item()** is used to verify that an item is in the container's item hashes.

In [8]:
from src.groups.base.value import BaseValue

hello_item: BaseValue = container_one.items[0]

if container_two._verify_item(hello_item):
    print('"hello" Item is verified in container_two. 👍')

world_item: BaseValue = container_one.items[1]

if not container_two._verify_item(world_item):
    print('"world" Item is not verified in container_two. 👍')

if container_one._hash_type() == container_two._hash_type():
    print('Containers are the same type. 👍')

"hello" Item is verified in container_two. 👍
"world" Item is not verified in container_two. 👍
Containers are the same type. 👍
